# Candlestick Visualization

In [1]:
import pandas as pd
import numpy as np
import requests
from stockstats import StockDataFrame as sdf
import bokeh.plotting as bk
from bokeh.models import Label, HoverTool, BoxZoomTool, PanTool, ZoomInTool, ZoomOutTool, ResetTool, BooleanFilter, LinearAxis, CDSView, DataRange1d, ColumnDataSource
API_URL = 'https://api.iextrading.com/1.0'

In [2]:
res = requests.get(f'{API_URL}/stock/MSFT/chart/5y')

In [3]:
data = res.json()

In [4]:
df = pd.DataFrame(data)

In [5]:
df.sample(2)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap
216,-0.516034,0.277568,-1.223,41.6901,2014-10-30,41.9345,"Oct 30, 14",41.4366,41.9345,30073873,30073873,41.6695
18,-0.452242,-0.011412,-1.382,32.2600,2014-01-17,32.6590,"Jan 17, 14",32.0560,32.6590,46298736,46298736,32.3978


In [6]:
seqs = np.arange(df.shape[0])
df['seqs'] = pd.Series(seqs)

In [7]:
df.sample(2)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap,seqs
41,0.205483,0.039804,0.609,33.9313,2014-02-21,34.2619,"Feb 21, 14",33.8241,33.8956,38030656,38030656,34.0174,41
2,0.407907,0.007609,1.256,32.8807,2013-12-24,32.9605,"Dec 24, 13",32.4905,32.5615,14242997,14242997,32.8248,2


In [8]:
df['changePercent'] = df['changePercent'].apply(lambda x: str(x) + '%')

In [9]:
df['mid'] = df.apply(lambda x: (x['open'] + x['close']) / 2, axis=1)

In [10]:
df['height'] = df.apply(
    lambda x: x['close'] - x['open'] if x['close'] != x['open'] else 0.01, axis=1)

In [11]:
inc = df.close > df.open
dec = df.close < df.open
w = .3

In [12]:
sourceInc = bk.ColumnDataSource(df.loc[inc])
sourceDec = bk.ColumnDataSource(df.loc[dec])

In [13]:
hover = HoverTool(
    tooltips=[
        ('Date', '@date'),
        ('Low', '@low'),
        ('High', '@high'),
        ('Open', '@open'),
        ('Close', '@close'),
        ('Percent', '@changePercent'),
    ]
)

In [14]:
TOOLS = [hover, BoxZoomTool(), PanTool(), ZoomInTool(), ZoomOutTool(), ResetTool()]

In [15]:
p = bk.figure(plot_width=1200, plot_height=800, title='Microsoft', tools=TOOLS, toolbar_location='above')

In [16]:
p.xaxis.major_label_orientation = np.pi/4
p.grid.grid_line_alpha = w

In [17]:
descriptor = Label(x=70, y=70, text='cool graph')
p.add_layout(descriptor)

In [18]:
p.segment(df.seqs[inc], df.high[inc], df.seqs[inc], df.low[inc], color='green')

GlyphRenderer(id='1043', ...)

In [19]:
p.segment(df.seqs[dec], df.high[dec], df.seqs[dec], df.low[dec], color='red')

GlyphRenderer(id='1050', ...)

In [20]:
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='red', line_color='red', source=sourceDec)

GlyphRenderer(id='1057', ...)

In [21]:
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='green', line_color='green', source=sourceInc)

GlyphRenderer(id='1064', ...)

In [22]:
# bk.show(p)

# MACD with Histogram adapted from https://github.com/pedromartins4/bokeh-stock-market/blob/master/run.py

In [23]:
new_data = sdf(df.copy())
stock = ColumnDataSource(data=dict(macd=[], macd_signal=[], date=[], OBV=[], macd_histogram=[], zeros=[]))
stock.data['macd_histogram'] = new_data['macdh']
stock.data['macd'] = new_data['macd']
stock.data['macd_signal'] = new_data['macds']
stock.data['date'] = pd.to_datetime(df['date'])

obv = []
for today in range(0, len(new_data)):
    if today == 0:
        obv.append(0)
        continue
    if new_data.close.iloc[today] > new_data.close.iloc[today - 1]:
        obv.append(obv[today - 1] + new_data.volume.iloc[today])
        continue
    if new_data.close.iloc[today] < new_data.close.iloc[today - 1]:
        obv.append(obv[today - 1] - new_data.volume.iloc[today])
        continue
    obv.append(obv[today - 1])
stock.data['OBV'] = np.array(obv)
stock.data['zeros'] = np.zeros(len(obv))



In [24]:
new = bk.figure(x_axis_type='datetime', plot_width=1200, plot_height=200, title='Moving Average Convergence/Divergence with Histogram', tools=TOOLS, toolbar_location='above')

In [25]:
up = [True if val > 0 else False for val in stock.data['macd_histogram']]
down = [True if val < 0 else False for val in stock.data['macd_histogram']]

In [26]:
view_upper = CDSView(source=stock, filters=[BooleanFilter(up)])
view_lower = CDSView(source=stock, filters=[BooleanFilter(down)])

In [27]:
new.vbar(x='date', top='macd_histogram', bottom='zeros', width=30000000, color='green', source=stock, view=view_upper)
new.vbar(x='date', top='zeros', bottom='macd_histogram', width=30000000, color='red', source=stock, view=view_lower)

GlyphRenderer(id='1110', ...)

In [28]:
new.extra_y_ranges = {'macd' : DataRange1d()}
new.add_layout(LinearAxis(y_range_name='macd'), 'right')

In [29]:
new.line(x='date', y='macd', line_width=2, color='blue', source=stock, legend='MACD', muted_color='blue', muted_alpha=0, y_range_name='macd')
new.line(x='date', y='macd_signal', line_width=2, color='lightblue', source=stock, legend='Signal', muted_color='lightblue', muted_alpha=0, y_range_name='macd')

GlyphRenderer(id='1149', ...)

In [30]:
new.legend.location = 'bottom_left'
new.legend.border_line_alpha = 0
new.legend.background_fill_alpha = 0
new.legend.click_policy = 'mute'

In [31]:
new.yaxis.ticker = []
new.yaxis.axis_line_alpha = 0

In [32]:
bk.show(new)